In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def batch_with_dynamic_pad_not_shuffled():
    def im_process(raw_image):
        image = tf.decode_raw(raw_image, tf.uint8)
        image = tf.reshape(image, [64, 64, 3])
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        image = tf.clip_by_value(image, 0.0, 1.0)
        image = tf.multiply(tf.subtract(image, 0.5), 2)
        return image

    reader = tf.TFRecordReader()
    path_to_tfrecords_file = '/home/amax/Documents/wit/data/val.tfrecords'
    batch_size = 128
    num_reader_threads = 4
    num_preprocess_threads = 4
    min_queue_examples = 5000

    filename_queue = tf.train.string_input_producer(
        [path_to_tfrecords_file], shuffle=False, name='filename_queue')

    capacity = min_queue_examples + 3 * batch_size  # 3->10
    values_queue = tf.FIFOQueue(
        capacity=capacity, dtypes=[tf.string], name="fifo_input_queue_name")

    enqueue_ops = []
    for _ in range(num_reader_threads):
        _, value = reader.read(filename_queue)
        enqueue_ops.append(values_queue.enqueue([value]))

    tf.train.queue_runner.add_queue_runner(tf.train.queue_runner.QueueRunner(
        values_queue, enqueue_ops))

    enqueue_list = []
    for _ in range(num_preprocess_threads):
        serialized_example = values_queue.dequeue()

        features = tf.parse_single_example(
            serialized_example,
            features={
                'image': tf.FixedLenFeature([], tf.string),
                'length': tf.FixedLenFeature([], tf.int64),
                'digits': tf.FixedLenFeature([6], tf.int64)
            })
        # data process
        image = im_process(features['image'])
        enqueue_list.append([image])

    queue_capacity = 2 * num_preprocess_threads * batch_size  # 2->10
    images = tf.train.batch_join(
        enqueue_list,
        batch_size=batch_size,
        capacity=queue_capacity,
        dynamic_pad=True,
        name="batch_not_shuffled")
    images = tf.image.resize_images(images, [54, 54])

    return images

In [3]:
images = batch_with_dynamic_pad_not_shuffled()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    outputs = sess.run([images])

    coord.request_stop()
    coord.join(threads)
print(outputs)

[array([[[[-0.34901959, -0.38823527, -0.53725487],
         [-0.34901959, -0.39114013, -0.53870732],
         [-0.34901959, -0.40392154, -0.54509801],
         ..., 
         [ 0.05882359, -0.04313719, -0.17647052],
         [ 0.0538853 , -0.03819891, -0.17647052],
         [ 0.05098045, -0.03529406, -0.17647052]],

        [[-0.36063904, -0.39694986, -0.54306459],
         [-0.36037007, -0.39958575, -0.54505497],
         [-0.35918662, -0.41118371, -0.55381262],
         ..., 
         [ 0.06027602, -0.04168476, -0.17501809],
         [ 0.05533773, -0.03674648, -0.17501809],
         [ 0.05243288, -0.03384162, -0.17501809]],

        [[-0.41176468, -0.43529409, -0.56862742],
         [-0.41031224, -0.43674654, -0.5729847 ],
         [-0.40392154, -0.44313723, -0.59215683],
         ..., 
         [ 0.06666672, -0.03529406, -0.16862738],
         [ 0.06172844, -0.03035577, -0.16862738],
         [ 0.05882359, -0.02745092, -0.16862738]],

        ..., 
        [[-0.04313719, -0.13725483